In [5]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd
from sklearn.metrics import f1_score, classification_report

# 모델과 토크나이저 로드
model_path = r"C:\Users\user\OneDrive\바탕 화면\project\OSS_Project\AI\text-model\models\bert_model"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 테스트 데이터 로드
data_path = r"C:\Users\user\OneDrive\바탕 화면\project\OSS_Project\AI\text-model\data\processed_data.csv"
data = pd.read_csv(data_path)

# 입력 문장 토크나이징
inputs = tokenizer(list(data['입력 문장']), padding=True, truncation=True, return_tensors="pt")

# 도수 레이블 매핑
도수_매핑 = {'낮은': 0, '중간': 1, '높은': 2}
labels = torch.tensor([도수_매핑[도수] for 도수 in data['도수']])

# 데이터셋 생성
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels)
dataloader = DataLoader(dataset, batch_size=2)

# 모델 평가
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        all_preds.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# F1 스코어 및 분류 리포트 출력
f1 = f1_score(all_labels, all_preds, average="weighted")
print(f"Weighted F1 Score: {f1}")

print("\nClassification Report:")
label_names = ['낮은', '중간', '높은']
print(classification_report(all_labels, all_preds, target_names=label_names))

OSError: Incorrect path_or_model_id: 'models/bert_model/'. Please provide either the path to a local folder or the repo_id of a model on the Hub.